In [2]:
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import json

In [46]:
class Image_Factory(object):
    '''
    The Image_Factory takes a Composite_Frame object and a directory to save images
    in and populates the directory with heat map images derived from network resolutions
    in the Composite_Frame
    '''
    def __init__(self, csv_path: Path, img_path: Path):
        '''
        Instance Variables:
            @self.img_path -> path object to write heat maps too
            @self.label -> label to use in naming images
            @self.img_limit -> how many images to create 
        '''
        self.img_path = img_path
        self.label = str(img_path).split('/')[-1]
        self.img_count = 0

        # Extract configurations
        with open('data/stats/img_factory_confs.json', 'r') as fin:
            self.confs = json.load(fin)

        self._create_heatmaps(csv_path)

    def _create_heatmaps(self, csv_path: Path):
        addresses = set(self.confs['addresses'])

        _, ax = plt.subplots(1, 1, figsize=(1, 1), dpi=23)

        for fin in csv_path.iterdir():
            frame = pd.read_csv(fin, usecols=['saddr', 'stime', 'ltime', 'tbytes'],

                                dtype={'saddr': str, 
                                        'stime': np.float, 
                                        'ltime': np.float,
                                        'tbytes': np.int
                                        }
                                )
            missing_addrs = list(addresses - set(list(frame.saddr.unique().tolist())))
            for addr in missing_addrs:
                frame = frame.append({'saddr': addr,
                              'stime': 0.,
                              'ltime': 0.,
                              'tbytes': 0}, ignore_index=True)

            frame = frame.filter(['saddr', 'tbytes']).groupby('saddr').sum().reset_index()

            frame = pd.pivot_table(frame, values='tbytes', index='saddr', columns=None)

            _ = sns.heatmap(frame, xticklabels=False, yticklabels=False, cbar=False)

            ax.set_ylabel('')
            ax.set_xlabel('')

            img_name = self.label + str(self.img_count)
            plt.savefig(self.img_path / img_name)

            self.img_count += 1


In [47]:
csv_path = Path.cwd() / 'data' / 'transformed_data'/ 'clean_tcp' 
img_path = Path.cwd() / 'data' / 'heat_maps' / 'clean_tcp'

img_fact = Image_Factory(csv_path, img_path) 

In [48]:
csv_path = Path.cwd() / 'data' / 'transformed_data' / 'ddos_tcp' 
img_path = Path.cwd() / 'data' / 'heat_maps' / 'ddos_tcp'

img_fact = Image_Factory(csv_path, img_path) 